# Passive Agressive Hyperparameter Tunning

We decided to do a separate hyperparameter tunning with this classifiers, since the clasification graphic showed a good score.

In [1]:
from sklearn.linear_model import PassiveAggressiveClassifier
%matplotlib notebook
import IPython
from IPython.display import display
import pandas as pd
import csv
from numpy import nan as NA
from datetime import datetime
import re
import sys
import numpy as np
import matplotlib.pyplot as plt 
from pandas import *
import pickle
import requests
import os
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from yellowbrick.classifier import ClassificationReport
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
with open('x_y_z.pickle', 'rb') as xyz:
    feat_var = pickle.load(xyz)
X =  feat_var[0]
y =  feat_var[1] #y is "offense group"
z =  feat_var[2] # z is "ucr-rank"
z_i = pd.Series(z)
z_i.value_counts()

6    102462
7     82325
4     26379
5     23033
8     22602
3     17686
2      2170
1       942
9       196
Name: ucr-rank, dtype: int64

In [3]:
X = X[["hour","street", "month", "day", "LATITUDE", "LONGITUDE", "Temperature"]]

In [4]:
y_i = pd.Series(y)
y_i.value_counts()  # 0 non violent, 1 violent

0    230618
1     47177
dtype: int64

In [5]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_sample(X, y)

In [6]:
X_res_i = pd.DataFrame(X_res)
y_res_i = pd.Series(y_res)

print (X_res_i.info())
print (y_res_i.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461236 entries, 0 to 461235
Data columns (total 7 columns):
0    461236 non-null int64
1    461236 non-null int64
2    461236 non-null int64
3    461236 non-null int64
4    461236 non-null int64
5    461236 non-null int64
6    461236 non-null int64
dtypes: int64(7)
memory usage: 24.6 MB
None
1    230618
0    230618
dtype: int64


In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_res, y_res, test_size=0.2,random_state= 0)
scaler = StandardScaler()
#scaler =  MinMaxScaler()
#scaler = Normalizer()
X_train = scaler.fit(X_train).transform(X_train)
X_test = scaler.fit(X_test).transform(X_test)

c:\users\franc\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
def gridsearch(model, parameters):
    grid_search = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'accuracy', cv = 4, n_jobs = -1, verbose = 10)
    grid_search = grid_search.fit(X_train, y_train)
    accuracy = grid_search.best_score_
    best_params = grid_search.best_params_
    return accuracy, best_params


In [9]:
list_params = [{'C':[1, 3, 5, 10, 30, 50, 100, 200], 'fit_intercept': [True, False], 'max_iter': [5,10,20,50, 100]}]


In [10]:
scores_dict = {'accuracy': [], 'best_params': []}

if __name__ == '__main__':
    pas = PassiveAggressiveClassifier()
    acu, best_params = gridsearch(pas, list_params)
    scores_dict['accuracy'].append(acu)
    scores_dict['best_params'].append(best_params)

Fitting 4 folds for each of 80 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:  1.4min
[Paralle

In [11]:
print(scores_dict)

{'accuracy': [0.5735525274534673], 'best_params': [{'C': 1, 'fit_intercept': False, 'max_iter': 10}]}
